In [3]:
import bs4 as bs
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

In [6]:
#Chrome
chrome_options = Options()

#opens the website
driver = webdriver.Chrome(ChromeDriverManager().install()) #gets the new version of chrome 
driver.maximize_window()

url = "https://forums.edmunds.com/discussion/2444/general/x/what-car-is-right-for-me-help-me-choose"
driver.get(url)


#function to click next 
def click_next():
    try: 
        element = driver.find_element_by_css_selector("[title*='Next Page']")
        element.click()
    except IndexError: 
        print("Next button not found")



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\carlo\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
<ipython-input-6-6d50d586a1df>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install()) #gets the new version of chrome


In [7]:
messages_list = []
users_list = []
dates_list = []
nbposts_list = []
roles_list = []

num_pages = 84

for page_count in range(1,num_pages):
    time.sleep(1)    
    page = driver.page_source
    soup = BeautifulSoup(page,'lxml')

    messages = soup.find_all('div', class_='Message userContent')
    user_ids = soup.find_all('a', class_='Username js-userCard')
    dates = soup.find_all('span', class_='MItem DateCreated')
    roles = soup.find_all('span', class_='MItem RoleTitle')
    nbposts = soup.find_all('span', class_='MItem PostCount')
    

    #appending messages 
    for i in messages:
            messages_list.append(i.get_text())

    #appending dates
    for i in dates:
            dates_list.append(i.get_text())
            
    #appending users
    for i in user_ids: 
        users_list.append(i.get_text())
        
    #appending user nb of posts
    for i in nbposts: 
        nbposts_list.append(i.get_text())
    
    #appending member role
    for i in roles: 
        roles_list.append(i.get_text())
    

    print("Done scraping", page_count, "of 84 pages.")
    click_next()
    time.sleep(1)
    
df = pd.DataFrame({"Date": dates_list, "User_Id": users_list, "Message": messages_list, "NumberOfPastPosts" : nbposts_list, "Role" : roles_list})
df.to_csv('data.csv')


Done scraping 1 of 84 pages.


<ipython-input-6-6d50d586a1df>:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_css_selector("[title*='Next Page']")


Done scraping 2 of 84 pages.
Done scraping 3 of 84 pages.
Done scraping 4 of 84 pages.
Done scraping 5 of 84 pages.
Done scraping 6 of 84 pages.
Done scraping 7 of 84 pages.
Done scraping 8 of 84 pages.
Done scraping 9 of 84 pages.
Done scraping 10 of 84 pages.
Done scraping 11 of 84 pages.
Done scraping 12 of 84 pages.
Done scraping 13 of 84 pages.
Done scraping 14 of 84 pages.
Done scraping 15 of 84 pages.
Done scraping 16 of 84 pages.
Done scraping 17 of 84 pages.
Done scraping 18 of 84 pages.
Done scraping 19 of 84 pages.
Done scraping 20 of 84 pages.
Done scraping 21 of 84 pages.


KeyboardInterrupt: 

In [4]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

df = pd.read_csv("data.csv")
df = df.drop(['Unnamed: 0'],axis=1)

df

,Date,User_Id,Message,NumberOfPastPosts,Role
0,\nSeptember 2001,pat,\nNeed help choosing your next vehicle? Tell u...,"Posts: 10,421",Member
1,\nSeptember 2001,willow25,"\nI'm buying a ""new"" car by the end of the yea...",Posts: 2,Member
2,\nOctober 2001,dindak,\nBuy a 2001/ 2002 Oldsmobile Intrigue. It's o...,"Posts: 6,632",Member
3,\nOctober 2001,peteri1,\nAm looking at 1 of 4 choices to replace my 9...,Posts: 21,Member
4,\nOctober 2001,mrdetailer,\nGenerally a 3-4 year old car range is a good...,"Posts: 1,118",Member
...,...,...,...,...,...
4146,\nFebruary 2021,kyfdx,\nqbrozen said: show previous quoteskyfdx said...,"Posts: 202,806",Moderator
4147,\nFebruary 2021,backy,\nKamCottage said:Thank you backy. Since I fir...,"Posts: 18,946",Member
4148,\nFebruary 2021,RayeEliza,\nI am doing major research in trying to find ...,Posts: 6,Member
4149,\nFebruary 2021,mlevine,\nI would lean toward subaru. Need to see whic...,Posts: 512,Member


In [5]:
#cleaning
#dates
from datetime import datetime
df['Date'] = df['Date'].map(lambda x: x.lstrip('\n'))
df['Date'] = pd.to_datetime(df['Date'])

#messages
df['Message'] = df['Message'].map(lambda x: x.lstrip('\n'))
#nb posts
df['NumberOfPastPosts'] = df['NumberOfPastPosts'].map(lambda x: x.lstrip('Posts: '))

df

,Date,User_Id,Message,NumberOfPastPosts,Role
0,2001-09-01,pat,Need help choosing your next vehicle? Tell us ...,"10,421",Member
1,2001-09-01,willow25,"I'm buying a ""new"" car by the end of the year ...",2,Member
2,2001-10-01,dindak,Buy a 2001/ 2002 Oldsmobile Intrigue. It's one...,"6,632",Member
3,2001-10-01,peteri1,Am looking at 1 of 4 choices to replace my 99G...,21,Member
4,2001-10-01,mrdetailer,Generally a 3-4 year old car range is a good o...,"1,118",Member
...,...,...,...,...,...
4146,2021-02-01,kyfdx,qbrozen said: show previous quoteskyfdx said:I...,"202,806",Moderator
4147,2021-02-01,backy,KamCottage said:Thank you backy. Since I first...,"18,946",Member
4148,2021-02-01,RayeEliza,I am doing major research in trying to find my...,6,Member
4149,2021-02-01,mlevine,I would lean toward subaru. Need to see which ...,512,Member


In [6]:
df

,Date,User_Id,Message,NumberOfPastPosts,Role
0,2001-09-01,pat,Need help choosing your next vehicle? Tell us ...,"10,421",Member
1,2001-09-01,willow25,"I'm buying a ""new"" car by the end of the year ...",2,Member
2,2001-10-01,dindak,Buy a 2001/ 2002 Oldsmobile Intrigue. It's one...,"6,632",Member
3,2001-10-01,peteri1,Am looking at 1 of 4 choices to replace my 99G...,21,Member
4,2001-10-01,mrdetailer,Generally a 3-4 year old car range is a good o...,"1,118",Member
...,...,...,...,...,...
4146,2021-02-01,kyfdx,qbrozen said: show previous quoteskyfdx said:I...,"202,806",Moderator
4147,2021-02-01,backy,KamCottage said:Thank you backy. Since I first...,"18,946",Member
4148,2021-02-01,RayeEliza,I am doing major research in trying to find my...,6,Member
4149,2021-02-01,mlevine,I would lean toward subaru. Need to see which ...,512,Member


In [7]:
#defining the text processing method to eliminate punctuation and stopwords

import string

#importing stopwords from the natural language toolkit library 
from nltk.corpus import stopwords

def text_process(mess):
    nopunc =[char for char in mess if char not in string.punctuation]
    nopunc=''.join(nopunc)
    final_list = []
    for word in nopunc.split():
        if word.lower() not in stopwords.words('english'):
            final_list.append(word)
    return final_list

df['Message_words'] = df['Message'].apply(text_process)

In [8]:
df

,Date,User_Id,Message,NumberOfPastPosts,Role,Message_words
0,2001-09-01,pat,Need help choosing your next vehicle? Tell us ...,"10,421",Member,"[Need, help, choosing, next, vehicle, Tell, us..."
1,2001-09-01,willow25,"I'm buying a ""new"" car by the end of the year ...",2,Member,"[Im, buying, new, car, end, year, first, choic..."
2,2001-10-01,dindak,Buy a 2001/ 2002 Oldsmobile Intrigue. It's one...,"6,632",Member,"[Buy, 2001, 2002, Oldsmobile, Intrigue, one, b..."
3,2001-10-01,peteri1,Am looking at 1 of 4 choices to replace my 99G...,21,Member,"[looking, 1, 4, choices, replace, 99GS400, Ive..."
4,2001-10-01,mrdetailer,Generally a 3-4 year old car range is a good o...,"1,118",Member,"[Generally, 34, year, old, car, range, good, o..."
...,...,...,...,...,...,...
4146,2021-02-01,kyfdx,qbrozen said: show previous quoteskyfdx said:I...,"202,806",Moderator,"[qbrozen, said, show, previous, quoteskyfdx, s..."
4147,2021-02-01,backy,KamCottage said:Thank you backy. Since I first...,"18,946",Member,"[KamCottage, saidThank, backy, Since, first, p..."
4148,2021-02-01,RayeEliza,I am doing major research in trying to find my...,6,Member,"[major, research, trying, find, new, car, lean..."
4149,2021-02-01,mlevine,I would lean toward subaru. Need to see which ...,512,Member,"[would, lean, toward, subaru, Need, see, model..."


In [9]:
import joblib 

spam_model = joblib.load('spam_detector_model.pkl')
spam_xtrain = joblib.load('xtrain.pkl')
spam_ytrain = joblib.load('ytrain.pkl')

In [10]:
#importing our model and methods

#we choose to work with a multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB

#CountVectorizer converts a collection of text documents to a matrix of token counts
from sklearn.feature_extraction.text import CountVectorizer

#The TfidfTransformer converts a collection of documents to a matrix of TF-IDF features. 
# TFIDF is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus
from sklearn.feature_extraction.text import TfidfTransformer

#pipeline creation to accelerate sample predictions

from sklearn.pipeline import Pipeline

spam_pipeline = Pipeline([
   ( 'bow',CountVectorizer(analyzer=text_process)),
    ('tfidf',TfidfTransformer()),
    ('classifier',spam_model),
])


spam_pipeline.fit(spam_xtrain,spam_ytrain)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x0000018E7755A3A0>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB(alpha=0.5))])

In [11]:
def ModerateMyPost(post):
    sample_spam_predict = spam_pipeline.predict([post])
    my_mod = {'label' : str(sample_spam_predict) }
    return my_mod

In [12]:
pd.set_option('display.max_rows', 50)


df_notnan = df[df['Message'].notna()]


labels = []
i=0
for post in df_notnan.Message:
    sample_spam_predict = spam_pipeline.predict([post])
    labels.append(sample_spam_predict)
    print('Checked message', i, ' of', len(df_notnan.Message))
    i +=1

Checked message 0  of 4151
Checked message 1  of 4151
Checked message 2  of 4151
Checked message 3  of 4151
Checked message 4  of 4151
Checked message 5  of 4151
Checked message 6  of 4151
Checked message 7  of 4151
Checked message 8  of 4151
Checked message 9  of 4151
Checked message 10  of 4151
Checked message 11  of 4151
Checked message 12  of 4151
Checked message 13  of 4151
Checked message 14  of 4151
Checked message 15  of 4151
Checked message 16  of 4151
Checked message 17  of 4151
Checked message 18  of 4151
Checked message 19  of 4151
Checked message 20  of 4151
Checked message 21  of 4151
Checked message 22  of 4151
Checked message 23  of 4151
Checked message 24  of 4151
Checked message 25  of 4151
Checked message 26  of 4151
Checked message 27  of 4151
Checked message 28  of 4151
Checked message 29  of 4151
Checked message 30  of 4151
Checked message 31  of 4151
Checked message 32  of 4151
Checked message 33  of 4151
Checked message 34  of 4151
Checked message 35  of 4151
Ch

Checked message 288  of 4151
Checked message 289  of 4151
Checked message 290  of 4151
Checked message 291  of 4151
Checked message 292  of 4151
Checked message 293  of 4151
Checked message 294  of 4151
Checked message 295  of 4151
Checked message 296  of 4151
Checked message 297  of 4151
Checked message 298  of 4151
Checked message 299  of 4151
Checked message 300  of 4151
Checked message 301  of 4151
Checked message 302  of 4151
Checked message 303  of 4151
Checked message 304  of 4151
Checked message 305  of 4151
Checked message 306  of 4151
Checked message 307  of 4151
Checked message 308  of 4151
Checked message 309  of 4151
Checked message 310  of 4151
Checked message 311  of 4151
Checked message 312  of 4151
Checked message 313  of 4151
Checked message 314  of 4151
Checked message 315  of 4151
Checked message 316  of 4151
Checked message 317  of 4151
Checked message 318  of 4151
Checked message 319  of 4151
Checked message 320  of 4151
Checked message 321  of 4151
Checked messag

Checked message 581  of 4151
Checked message 582  of 4151
Checked message 583  of 4151
Checked message 584  of 4151
Checked message 585  of 4151
Checked message 586  of 4151
Checked message 587  of 4151
Checked message 588  of 4151
Checked message 589  of 4151
Checked message 590  of 4151
Checked message 591  of 4151
Checked message 592  of 4151
Checked message 593  of 4151
Checked message 594  of 4151
Checked message 595  of 4151
Checked message 596  of 4151
Checked message 597  of 4151
Checked message 598  of 4151
Checked message 599  of 4151
Checked message 600  of 4151
Checked message 601  of 4151
Checked message 602  of 4151
Checked message 603  of 4151
Checked message 604  of 4151
Checked message 605  of 4151
Checked message 606  of 4151
Checked message 607  of 4151
Checked message 608  of 4151
Checked message 609  of 4151
Checked message 610  of 4151
Checked message 611  of 4151
Checked message 612  of 4151
Checked message 613  of 4151
Checked message 614  of 4151
Checked messag

Checked message 873  of 4151
Checked message 874  of 4151
Checked message 875  of 4151
Checked message 876  of 4151
Checked message 877  of 4151
Checked message 878  of 4151
Checked message 879  of 4151
Checked message 880  of 4151
Checked message 881  of 4151
Checked message 882  of 4151
Checked message 883  of 4151
Checked message 884  of 4151
Checked message 885  of 4151
Checked message 886  of 4151
Checked message 887  of 4151
Checked message 888  of 4151
Checked message 889  of 4151
Checked message 890  of 4151
Checked message 891  of 4151
Checked message 892  of 4151
Checked message 893  of 4151
Checked message 894  of 4151
Checked message 895  of 4151
Checked message 896  of 4151
Checked message 897  of 4151
Checked message 898  of 4151
Checked message 899  of 4151
Checked message 900  of 4151
Checked message 901  of 4151
Checked message 902  of 4151
Checked message 903  of 4151
Checked message 904  of 4151
Checked message 905  of 4151
Checked message 906  of 4151
Checked messag

Checked message 1155  of 4151
Checked message 1156  of 4151
Checked message 1157  of 4151
Checked message 1158  of 4151
Checked message 1159  of 4151
Checked message 1160  of 4151
Checked message 1161  of 4151
Checked message 1162  of 4151
Checked message 1163  of 4151
Checked message 1164  of 4151
Checked message 1165  of 4151
Checked message 1166  of 4151
Checked message 1167  of 4151
Checked message 1168  of 4151
Checked message 1169  of 4151
Checked message 1170  of 4151
Checked message 1171  of 4151
Checked message 1172  of 4151
Checked message 1173  of 4151
Checked message 1174  of 4151
Checked message 1175  of 4151
Checked message 1176  of 4151
Checked message 1177  of 4151
Checked message 1178  of 4151
Checked message 1179  of 4151
Checked message 1180  of 4151
Checked message 1181  of 4151
Checked message 1182  of 4151
Checked message 1183  of 4151
Checked message 1184  of 4151
Checked message 1185  of 4151
Checked message 1186  of 4151
Checked message 1187  of 4151
Checked me

...

Checked message 2817  of 4151
Checked message 2818  of 4151
Checked message 2819  of 4151
Checked message 2820  of 4151
Checked message 2821  of 4151
Checked message 2822  of 4151
Checked message 2823  of 4151
Checked message 2824  of 4151
Checked message 2825  of 4151
Checked message 2826  of 4151
Checked message 2827  of 4151
Checked message 2828  of 4151
Checked message 2829  of 4151
Checked message 2830  of 4151
Checked message 2831  of 4151
Checked message 2832  of 4151
Checked message 2833  of 4151
Checked message 2834  of 4151
Checked message 2835  of 4151
Checked message 2836  of 4151
Checked message 2837  of 4151
Checked message 2838  of 4151
Checked message 2839  of 4151
Checked message 2840  of 4151
Checked message 2841  of 4151
Checked message 2842  of 4151
Checked message 2843  of 4151
Checked message 2844  of 4151
Checked message 2845  of 4151
Checked message 2846  of 4151
Checked message 2847  of 4151
Checked message 2848  of 4151
Checked message 2849  of 4151
Checked me

Checked message 3096  of 4151
Checked message 3097  of 4151
Checked message 3098  of 4151
Checked message 3099  of 4151
Checked message 3100  of 4151
Checked message 3101  of 4151
Checked message 3102  of 4151
Checked message 3103  of 4151
Checked message 3104  of 4151
Checked message 3105  of 4151
Checked message 3106  of 4151
Checked message 3107  of 4151
Checked message 3108  of 4151
Checked message 3109  of 4151
Checked message 3110  of 4151
Checked message 3111  of 4151
Checked message 3112  of 4151
Checked message 3113  of 4151
Checked message 3114  of 4151
Checked message 3115  of 4151
Checked message 3116  of 4151
Checked message 3117  of 4151
Checked message 3118  of 4151
Checked message 3119  of 4151
Checked message 3120  of 4151
Checked message 3121  of 4151
Checked message 3122  of 4151
Checked message 3123  of 4151
Checked message 3124  of 4151
Checked message 3125  of 4151
Checked message 3126  of 4151
Checked message 3127  of 4151
Checked message 3128  of 4151
Checked me

Checked message 3372  of 4151
Checked message 3373  of 4151
Checked message 3374  of 4151
Checked message 3375  of 4151
Checked message 3376  of 4151
Checked message 3377  of 4151
Checked message 3378  of 4151
Checked message 3379  of 4151
Checked message 3380  of 4151
Checked message 3381  of 4151
Checked message 3382  of 4151
Checked message 3383  of 4151
Checked message 3384  of 4151
Checked message 3385  of 4151
Checked message 3386  of 4151
Checked message 3387  of 4151
Checked message 3388  of 4151
Checked message 3389  of 4151
Checked message 3390  of 4151
Checked message 3391  of 4151
Checked message 3392  of 4151
Checked message 3393  of 4151
Checked message 3394  of 4151
Checked message 3395  of 4151
Checked message 3396  of 4151
Checked message 3397  of 4151
Checked message 3398  of 4151
Checked message 3399  of 4151
Checked message 3400  of 4151
Checked message 3401  of 4151
Checked message 3402  of 4151
Checked message 3403  of 4151
Checked message 3404  of 4151
Checked me

Checked message 3651  of 4151
Checked message 3652  of 4151
Checked message 3653  of 4151
Checked message 3654  of 4151
Checked message 3655  of 4151
Checked message 3656  of 4151
Checked message 3657  of 4151
Checked message 3658  of 4151
Checked message 3659  of 4151
Checked message 3660  of 4151
Checked message 3661  of 4151
Checked message 3662  of 4151
Checked message 3663  of 4151
Checked message 3664  of 4151
Checked message 3665  of 4151
Checked message 3666  of 4151
Checked message 3667  of 4151
Checked message 3668  of 4151
Checked message 3669  of 4151
Checked message 3670  of 4151
Checked message 3671  of 4151
Checked message 3672  of 4151
Checked message 3673  of 4151
Checked message 3674  of 4151
Checked message 3675  of 4151
Checked message 3676  of 4151
Checked message 3677  of 4151
Checked message 3678  of 4151
Checked message 3679  of 4151
Checked message 3680  of 4151
Checked message 3681  of 4151
Checked message 3682  of 4151
Checked message 3683  of 4151
Checked me

Checked message 3931  of 4151
Checked message 3932  of 4151
Checked message 3933  of 4151
Checked message 3934  of 4151
Checked message 3935  of 4151
Checked message 3936  of 4151
Checked message 3937  of 4151
Checked message 3938  of 4151
Checked message 3939  of 4151
Checked message 3940  of 4151
Checked message 3941  of 4151
Checked message 3942  of 4151
Checked message 3943  of 4151
Checked message 3944  of 4151
Checked message 3945  of 4151
Checked message 3946  of 4151
Checked message 3947  of 4151
Checked message 3948  of 4151
Checked message 3949  of 4151
Checked message 3950  of 4151
Checked message 3951  of 4151
Checked message 3952  of 4151
Checked message 3953  of 4151
Checked message 3954  of 4151
Checked message 3955  of 4151
Checked message 3956  of 4151
Checked message 3957  of 4151
Checked message 3958  of 4151
Checked message 3959  of 4151
Checked message 3960  of 4151
Checked message 3961  of 4151
Checked message 3962  of 4151
Checked message 3963  of 4151
Checked me

In [13]:
pd.set_option('display.max_rows', 20)

df_notnan

,Date,User_Id,Message,NumberOfPastPosts,Role,Message_words
0,2001-09-01,pat,Need help choosing your next vehicle? Tell us ...,"10,421",Member,"[Need, help, choosing, next, vehicle, Tell, us..."
1,2001-09-01,willow25,"I'm buying a ""new"" car by the end of the year ...",2,Member,"[Im, buying, new, car, end, year, first, choic..."
2,2001-10-01,dindak,Buy a 2001/ 2002 Oldsmobile Intrigue. It's one...,"6,632",Member,"[Buy, 2001, 2002, Oldsmobile, Intrigue, one, b..."
3,2001-10-01,peteri1,Am looking at 1 of 4 choices to replace my 99G...,21,Member,"[looking, 1, 4, choices, replace, 99GS400, Ive..."
4,2001-10-01,mrdetailer,Generally a 3-4 year old car range is a good o...,"1,118",Member,"[Generally, 34, year, old, car, range, good, o..."
...,...,...,...,...,...,...
4146,2021-02-01,kyfdx,qbrozen said: show previous quoteskyfdx said:I...,"202,806",Moderator,"[qbrozen, said, show, previous, quoteskyfdx, s..."
4147,2021-02-01,backy,KamCottage said:Thank you backy. Since I first...,"18,946",Member,"[KamCottage, saidThank, backy, Since, first, p..."
4148,2021-02-01,RayeEliza,I am doing major research in trying to find my...,6,Member,"[major, research, trying, find, new, car, lean..."
4149,2021-02-01,mlevine,I would lean toward subaru. Need to see which ...,512,Member,"[would, lean, toward, subaru, Need, see, model..."


In [14]:
df_notnan['label'] = labels

In [15]:
for potential_spam in df_notnan[df_notnan['label'] == "spam"]['Message']:
    print("\n", potential_spam)


 97 Honda Accord 95K or 2003 Buick Century 71K both priced about $7900 and they have similar features 

 Get the Accord. 

 Here's the link: aramis885, "Nissan Altima Maintenance and Repair" #2163, 16 Jan 2008 3:43 pm. 

 If select trim Altima 2.5S + Premium Pkg, Is it DVD video playback?Or Only DVD audio playback?thanks. 

 Get a Camry. 

 The Kia, Fusion and Sonata are all $3,000-$4,000 below KBB fair price while the Accord is about $750 below. 


In [16]:
df_clean = df_notnan


joblib.dump(df_clean, "df_clean.pkl")

['df_clean.pkl']